# Importing the packages

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.multivariate.manova import MANOVA
import seaborn as sns
import statsmodels.api as sm

# Pre-analysis
## Basic variables

In [ ]:
dir_experiment1 = "../datasets/experiment1.csv"
dir_experiment2 = "../datasets/experiment2.csv"
dir_experiment3 = "../datasets/experiment3.csv"

dependandt = ["area_bbox", "x_displacement", "y_displacement", "x_error", "y_error", "elapsed_time", "fps"]
independant = ["x_speed", "y_speed", "method"]
plot_vars = dependandt + independant[:len(independant)-1]

type_dict = {"area_bbox" : 'float64', "x_displacement" : 'float64', "y_displacement": 'float64', "x_error" : 'float64', "y_error" : 'float64', "elapsed_time": 'float64', "fps": 'float64', "x_speed": 'float64', "y_speed": 'float64', "method": 'category'}

method_dict = {0: "", 1: "", 2: ""}

exp_dict = {0: "", 1: "", 2: ""}

alpha = 0.01

## Reading the datasets

In [ ]:
experiment1 = pd.read_csv(dir_experiment1, sep=",")
experiment2 = pd.read_csv(dir_experiment2, sep=",")
experiment3 = pd.read_csv(dir_experiment3, sep=",")
exp_list = [experiment1, experiment2, experiment3]

## Convert columns to the adequate type and drop NA's

In [ ]:
for i in range(0,len(exp_list)):
    exp_list[i] = exp_list[i].astype(type_dict)
    exp_list[i] = exp_list[i].dropna(0)

# Initial analysis

## Descriptive statistics

In [ ]:
for i in range(0,len(exp_list)):
    exp_list[i].describe()

## Scatter matrix

In [ ]:
for i in range(0, len(exp_list)):
    scatter = sns.pairplot(exp_list.loc[:,], hue="method")
    scatter = scatter.set(title=f"Scatter plot for experiment {exp_dict[i]}")
    scatter.savefig(f"scatter_{exp_dict[i]}.png", dpi=300)

## Normality check
### Shapiro-Wilkinson test

A Shapiro-Wilkinson test is performed to check the normality of the sample obtained

In [ ]:
for i in range(0, len(exp_list)):
    method1 = exp_list[i].loc[exp_list[i].loc["method"] == 1,:]
    print(f" - Results of the Shapiro-Wilkinson test for the experiment {exp_dict[i]} with the method {method_dict[0]}: \n            {stats.shapiro(method1)}")
    method2 = exp_list[i].loc[exp_list[i].loc["method"] == 2,:]
    print(f" - Results of the Shapiro-Wilkinson test for the experiment {exp_dict[i]} with the method {method_dict[1]}: \n            {stats.shapiro(method2)}")
    method3 = exp_list[i].loc[exp_list[i].loc["method"] == 3,:]
    print(f" - Results of the Shapiro-Wilkinson test for the experiment {exp_dict[i]} with the method {method_dict[2]}: \n            {stats.shapiro(method3)}")


### QQ-plot

In [ ]:
for i in range(0, len(exp_list)):
    m1 = exp_list[i].loc[exp_list[i].loc["method"] == 1,:]
    cov1 = np.cov(m1)
    icov1 = np.linalg.inv(cov1)
    left_term1 = np.dot((m1 - np.mean(m1)),icov1)
    total1 = np.dot(left_term1, (m1 - np.mean(m1)).T)
    mahalab1 = np.diag(total1)
    chi1 = np.random.chisquare(9, m1.shape[0])
    data1 = pd.DataFrame({"mahalab": mahalab1, "chi": chi1} )
    scatt1 = sns.scatterplot(x="chi", data=data1, y="mahalab")
    scatt1.set(title=f"QQ-plot for the experiment {exp_dict[i]} with the method {method_dict[0]}")
    scatt1.savefig(f"qqplot_exp{i}_m1.png", dpi=300)

    m2 = exp_list[i].loc[exp_list[i].loc["method"] == 2,:]
    cov2 = np.cov(m2)
    icov2 = np.linalg.inv(cov2)
    left_term2 = np.dot((m2 - np.mean(m2)),icov2)
    total2 = np.dot(left_term2, (m2 - np.mean(m2)).T)
    mahalab2 = np.diag(total2)
    chi2 = np.random.chisquare(9, m2.shape[0])
    data2 = pd.DataFrame({"mahalab": mahalab2, "chi": chi2} )
    scatt2 = sns.scatterplot(x="chi", data=data2, y="mahalab")
    scatt2.set(title=f"QQ-plot for the experiment {exp_dict[i]} with the method {method_dict[1]}")
    scatt2.savefig(f"qqplot_exp{i}_m2.png", dpi=300)

    m3 = exp_list[i].loc[exp_list[i].loc["method"] == 3,:]
    cov3 = np.cov(m3)
    icov3 = np.linalg.inv(cov3)
    left_term3 = np.dot((m3 - np.mean(m3)),icov3)
    total3 = np.dot(left_term3, (m3 - np.mean(m3)).T)
    mahalab3 = np.diag(total3)
    chi3 = np.random.chisquare(9, m3.shape[0])
    data3 = pd.DataFrame({"mahalab": mahalab3, "chi": chi3} )
    scatt3 = sns.scatterplot(x="chi", data=data3, y="mahalab")
    scatt3.set(title=f"QQ-plot for the experiment {exp_dict[i]} with the method {method_dict[2]}")
    scatt3.savefig(f"qqplot_exp{i}_m3.png", dpi=300)

## Tests
### MANOVA
Performing MANOVA test

### Performing a Barttlet test between both populations

#### With a Barttlet test we can check whether the covariances can be considered equal. In case they can be considered equal, a test will be performed to check whether the mean can also be considered equal or not in the case of equal covariances. Elsewhere, the test will be performed for the case of unequal covariances matrices.

In [ ]:
for i in range(0, len(exp_list)):
    ####################################################################################################
    ## Declare the datasets for each method
    m1 = exp_list[i].loc[exp_list[i].loc["method"] == 1,:]
    m2 = exp_list[i].loc[exp_list[i].loc["method"] == 2,:]
    m3 = exp_list[i].loc[exp_list[i].loc["method"] == 3,:]

    ####################################################################################################
    ## Obtain covariance and other relevant variables
    cov1 = np.cov(m1)
    cov2 = np.cov(m2)
    cov3 = np.cov(m3)

    n1 = m1.shape[0]
    n2 = m2.shape[0]
    n3 = m3.shape[0]
    p = cov1.shape[0]

    Sp12 = ((n1 - 1)*cov1 + (n2 - 1)*cov2)/(n1 + n2 - 2)
    Sp13 = ((n1 - 1)*cov1 + (n3 - 1)*cov3)/(n1 + n3 - 2)
    Sp23 = ((n3 - 1)*cov3 + (n2 - 1)*cov2)/(n3 + n2 - 2)

    equalCov12 = False
    equalCov13 = False
    equalCov23 = False

    ####################################################################################################
    ## Barttlet test
    c12 = 1 - ((2*p^2 + 3*p - 1)/(6*(p + 1))) * ((1/(n1-1)) + (1/(n1-1)) - (1/(n1+n2-1)))
    L12 = c12 * ((n1 + n2 - 2)*np.log(np.linalg.det(Sp12)) - (n1-1)*np.log(np.linalg.det(cov1)) - (n2-1)*np.log(np.linalg.det(cov2)))
    c23 = 1 - ((2*p^2 + 3*p - 1)/(6*(p + 1))) * ((1/(n2-1)) + (1/(n3-1)) - (1/(n3+n2-1)))
    L23 = c23 * ((n2 + n3 - 2)*np.log(np.linalg.det(Sp23)) - (n2-1)*np.log(np.linalg.det(cov2)) - (n3-1)*np.log(np.linalg.det(cov3)))
    c13 = 1 - ((2*p^2 + 3*p - 1)/(6*(p + 1))) * ((1/(n1-1)) + (1/(n3-1)) - (1/(n1+n3-1)))
    L13 = c13 * ((n1 + n3 - 2)*np.log(np.linalg.det(Sp13)) - (n1-1)*np.log(np.linalg.det(cov1)) - (n3-1)*np.log(np.linalg.det(cov3)))
    critical = stats.chi2(df=p*(p+1)/2).ppf(alpha)

    ####################################################################################################
    ## Printing the results
    if (L12 > critical):
        print(f"Rejecting H0 when comparing the methods 1 and 2.")
        equalCov12 = True
    else:
        print(f"Accepting H0 when comparing the methods 1 and 2.")

    if (L23 > critical):
        print(f"Rejecting H0 when comparing the methods 2 and 3.")
        equalCov23 = True
    else:
        print(f"Accepting H0 when comparing the methods 2 and 3.")

    if (L13 > critical):
        print(f"Rejecting H0 when comparing the methods 1 and 3.")
        equalCov13 = True
    else:
        print(f"Accepting H0 when comparing the methods 1 and 3.")
    ####################################################################################################
    ## Check for equal mean vector
    T12 = 0
    T13 = 0
    T23 = 0
    
